In [2]:
# styling definition
# 添加这一行以确保图片在 Notebook 中显示
%matplotlib inline
# 如果安装了 ipympl，也可以使用 %matplotlib widget 获得交互式绘图
# %matplotlib widget

import matplotlib.pyplot as plt
import scienceplots  # noqa: F401
import matplotlib as mpl
import warnings
import numpy as np

from scipy.optimize import curve_fit
from cycler import cycler


mpl.use("pgf")  # 注意：pgf 后端不显示图片，仅用于生成 LaTeX 导出文件。如果在 Notebook 中查看，请注释掉此行。

# 配置 PGF + XeLaTeX 环境
pgf_config = {
    "pgf.texsystem": "xelatex",  # 指定使用 xelatex 编译
    "text.usetex": True,  # 启用 LaTeX 渲染
    "font.family": "serif",  # 基本字体族
    "font.size": 8,  # 字号
    "pgf.rcfonts": False,  # 不使用 matplotlib 默认字体设置
    "pgf.preamble": r"""
        \usepackage{xeCJK}                    % 支持中文
        \usepackage{unicode-math}            % 支持 unicode 数学符号，如 −
        \setmainfont{Times New Roman}        % 英文字体
        \setCJKmainfont{SimSun}              % 中文字体
        \xeCJKsetup{CJKmath=true}            % 中文参与数学公式
    """,
}

# 应用配置
mpl.rcParams.update(pgf_config)

# Suppress warnings from polyfit for clarity
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

plt.style.use(["science"])

# 设置全局默认颜色循环 (Nature Publishing Group 风格)
mpl.rcParams["axes.prop_cycle"] = cycler(
    color=[
        "#E64B35",
        "#4DBBD5",
        "#00A087",
        "#3C5488",
        "#F39B7F",
        "#8491B4",
        "#91D1C2",
        "#DC0000",
        "#7E6148",
        "#B09C85",
    ]
)

In [3]:
# data
# 1. RC暂态
class data:
    pass

class result:
    pass

tau_theo = 1e3 * 0.1 # us
print("RC理论时间常数:", tau_theo, "us")

RC_data1 = data()
RC_data1.f = 800 # Hz
RC_data1.C = 0.1 # uF
RC_data1.R = 1e3 # 欧
RC_data1.E = 1.0 # V
RC_data1.T_half = 70.0 # us
RC_data1.tau_T_half = RC_data1.T_half / np.log(2) # us
print("根据半电压得到的时间常数:", RC_data1.tau_T_half, "us")
print("相对误差:", abs(RC_data1.tau_T_half - tau_theo) / tau_theo * 100, "%")
RC_data1.t = np.array([0, 38.0, 72.0, 128, 200, 242, 402, 632, 648, 676, 712, 812, 886, 1016, 1262]) #us, 632us大约为半个周期
RC_data1.t_up = RC_data1.t[0:8]
RC_data1.t_down = RC_data1.t[7:]
RC_data1.UC = np.array([0, 0.292, 0.496, 0.698, 0.854, 0.902, 0.984, 1.006, 0.904, 0.700, 0.496, 0.190, 0.0920, 0.0300, 0]) # V
RC_data1.UC_up = RC_data1.UC[0:8]
RC_data1.UC_down = RC_data1.UC[7:]

RC_data2 = data()
RC_data2.f = 2000 # Hz
RC_data2.C = 0.1 # uF
RC_data2.R = 1e3 # 欧
RC_data2.E = 1.0 # V
RC_data2.T_half = 66.0 # us
RC_data2.tau_T_half = RC_data2.T_half / np.log(2) # us
print("根据半电压得到的时间常数:", RC_data2.tau_T_half, "us")
print("相对误差:", abs(RC_data2.tau_T_half - tau_theo) / tau_theo * 100, "%")
RC_data2.t = np.array([0, 30.0, 66.0, 101, 121, 201, 251, 279, 304, 318, 367, 407, 501]) #us, 251us大约为半个周期
RC_data2.t_up = RC_data2.t[0:7]
RC_data2.t_down = RC_data2.t[6:]
RC_data2.UC = np.array([0, 0.220, 0.424, 0.564, 0.626, 0.788, 0.840, 0.628, 0.482, 0.426, 0.222, 0.128, 0]) # V
RC_data2.UC_up = RC_data2.UC[0:7]
RC_data2.UC_down = RC_data2.UC[6:]

# 方波信号从1变0
def RC_curve0(tau, t):
    return 1.006 * (1 - np.exp(-t / tau))

# 方波信号从0变1
def RC_curve1(tau, t):
    return 1.006 * np.exp(-t / tau)

# 2. RLC暂态
# 欠阻尼
RLC_data1 = data()
RLC_data1.L = 20.0 # mH
RLC_data1.C = 0.1 # uF
RLC_data1.f = 400 # Hz
RLC_data1.R = 100.0 # 欧
RLC_data1.E = 1.0 #V
RLC_data1.tau_theo = 2 * RLC_data1.L / RLC_data1.R * 1e3 # us
print("RLC理论时间常数:", RLC_data1.tau_theo, "us")
RLC_data1.omega_theo = (
    1
    / np.sqrt(RLC_data1.L * 1e-3 * RLC_data1.C * 1e-6)
    * np.sqrt(1 - (RLC_data1.R**2 * RLC_data1.C * 1e-6) / (4 * RLC_data1.L * 1e-3))
)  # rad/s
print("RLC理论角频率:", RLC_data1.omega_theo, "rad/s")
RLC_data1.t = np.array([0, 0.0750, 0.140, 0.220, 0.280, 0.365, 0.425, 0.500, 0.570, 0.650, 0.705, 0.780, 0.850, 0.995, 1.265, 1.340, 1.400, 1.480, 1.540, 1.620, 1.685, 1.760, 1.820, 1.900, 1.965, 2.105, 2.335, 2.500]) # ms，约1.265时为半个周期
RLC_data1.UC = np.array([0, 1.015, 1.560, 1.015, 0.730, 1.015, 1.170, 1.015, 0.935, 1.015, 1.055, 1.015, 0.995, 1.025, 1.015, 0, -0.530, 0, 0.290, 0, -0.150, 0, 0.0850, 0, -0.0400, 0.0300, 0, 0]) # V

# 临界阻尼
RLC_data2 = data()
RLC_data2.L = 20.0  # mH
RLC_data2.C = 0.1  # uF
RLC_data2.f = 400  # Hz
RLC_data2.R = 1000  # 欧
RLC_data2.E = 1.0  # V
RLC_data2.tau_theo = 2 * RLC_data2.L / RLC_data2.R * 1e3  # us
print("RLC理论时间常数:", RLC_data2.tau_theo, "us")
RLC_data2.t = np.array([0, 0.0700, 1.250, 1.330, 2.500])  # ms
RLC_data2.UC = np.array([0, 0.500, 1, 0.500, 0])  # V

# 过阻尼
RLC_data3 = data()
RLC_data3.L = 20.0  # mH
RLC_data3.C = 0.1  # uF
RLC_data3.f = 400  # Hz
RLC_data3.R = 3e3  # 欧
RLC_data3.E = 1.0  # V
RLC_data3.tau_theo = 2 * RLC_data3.L / RLC_data3.R * 1e3  # us
print("RLC理论时间常数:", RLC_data3.tau_theo, "us")
RLC_data3.t = np.array([0, 0.200, 1.250, 1.460, 2.500])  # ms
RLC_data3.UC = np.array([0, 0.974 / 2, 0.974, 0.974 / 2, 0])  # V

RC理论时间常数: 100.0 us
根据半电压得到的时间常数: 100.98865286222744 us
相对误差: 0.9886528622274398 %
根据半电压得到的时间常数: 95.21787269867158 us
相对误差: 4.782127301328416 %
RLC理论时间常数: 400.0 us
RLC理论角频率: 22220.486043288973 rad/s
RLC理论时间常数: 40.0 us
RLC理论时间常数: 13.333333333333334 us


In [4]:
# 对两个RC_data进行拟合，分别得到tau值，并计算相对误差

# Initialize global results storage
results_summary = {}

# 1. 定义拟合函数
# 充电过程 (0 -> V_max_source)
V_SOURCE = 1.006


def fit_func_charging(t, tau, max_v):
    return max_v * (1 - np.exp(-t / tau))


# 放电过程 (V_peak -> 0)
def fit_func_discharging(t, tau, v0):
    return v0 * np.exp(-t / tau)


# 计算 R^2 (决定系数)
def calc_r_squared(y_true, y_pred):
    residuals = y_true - y_pred
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)


# 通用拟合函数 (只计算，不绘图)
def analyze_rc_data(data_obj, zero_offset, max_offset, max_v):
    # --- 准备数据 ---
    t_up = data_obj.t_up
    u_up = data_obj.UC_up
    u_up[u_up == u_up.max()] -= max_offset  # 给充电数据中的最大值添加偏移
    t_down = data_obj.t_down
    u_down = data_obj.UC_down.copy()
    # 给放电数据中的0值添加偏移
    # u_down[u_down == u_down.min()] += zero_offset

    fit_func_charging_vmax = lambda t, tau: fit_func_charging(t, tau, max_v)  # noqa: E731

    # --- 充电拟合 ---
    popt_up, _ = curve_fit(fit_func_charging_vmax, t_up, u_up, p0=[100])
    tau_up = popt_up[0]

    # 计算充电 R^2
    u_up_pred = fit_func_charging_vmax(t_up, tau_up)
    r2_up = calc_r_squared(u_up, u_up_pred)

    # --- 放电拟合 ---
    # 时间平移：使放电开始时刻为 0
    t_down_shifted = t_down - t_down[0]
    v0_down = u_down[0]  # 放电初始电压

    fit_func_down_fixed_v0 = lambda t, tau: fit_func_discharging(t, tau, v0_down)  # noqa: E731

    popt_down, _ = curve_fit(fit_func_down_fixed_v0, t_down_shifted, u_down, p0=[100])
    tau_down = popt_down[0]

    # 计算放电 R^2
    u_down_pred = fit_func_down_fixed_v0(t_down_shifted, tau_down)
    r2_down = calc_r_squared(u_down, u_down_pred)

    return {
        "data_obj": data_obj,
        "tau_up": tau_up,
        "r2_up": r2_up,
        "tau_down": tau_down,
        "r2_down": r2_down,
        "v0_down": v0_down,
        "t_down_shifted": t_down_shifted,
        "t_down_start": t_down[0],
        "t_up": t_up,
        "t_down": t_down
    }

# 执行分析并保存结果
rc1_res = analyze_rc_data(RC_data1, zero_offset=0, max_offset=0, max_v=V_SOURCE)
rc2_res = analyze_rc_data(RC_data2, zero_offset=0, max_offset=0, max_v=1)

# Save to summary
results_summary['RC'] = {
    'rc1': rc1_res,
    'rc2': rc2_res
}

# 打印结果
for name, res in [("RC_data1", rc1_res), ("RC_data2", rc2_res)]:
    print(f"【{name}】")
    print(
        f"  [充电] tau: {res['tau_up']:.4f} us, R^2: {res['r2_up']:.6f}, 相对误差: {abs(res['tau_up'] - tau_theo) / tau_theo * 100:.2f}%"
    )
    print(
        f"  [放电] tau: {res['tau_down']:.4f} us, R^2: {res['r2_down']:.6f}, 相对误差: {abs(res['tau_down'] - tau_theo) / tau_theo * 100:.2f}%"
    )

【RC_data1】
  [充电] tau: 107.2682 us, R^2: 0.999870, 相对误差: 7.27%
  [放电] tau: 113.0574 us, R^2: 0.998493, 相对误差: 13.06%
【RC_data2】
  [充电] tau: 124.8907 us, R^2: 0.997896, 相对误差: 24.89%
  [放电] tau: 89.8021 us, R^2: 0.991558, 相对误差: 10.20%


In [5]:
# 对RLC欠阻尼情况进行拟合

# --- 1. 充电过程 (0 -> 1) ---
# 公式： U_C(t) = B + A * e^(-t / tau) * cos(omega * t - phi)
# 理论上 B=E=1, A 约为 -E (起始为0)

def fit_func_rlc(t, A, tau, omega, phi, B):
    return B + A * np.exp(-t / tau) * np.cos(omega * t - phi)

# 准备数据
t_rlc = RLC_data1.t
u_rlc = RLC_data1.UC

# 数据筛选：仅使用前半个周期 (t <= 1.265) 进行拟合
limit_t = 1.265
mask = t_rlc <= limit_t
t_rlc_fit = t_rlc[mask]
u_rlc_fit = u_rlc[mask]

# 初始猜测 (p0)
tau_guess = RLC_data1.tau_theo / 1000  # us -> ms
omega_guess = RLC_data1.omega_theo / 1000 # rad/s -> rad/ms

p0_rlc = [-1.0, tau_guess, omega_guess, 0, 1.0]

# 拟合充电
try:
    popt_rlc, _ = curve_fit(fit_func_rlc, t_rlc_fit, u_rlc_fit, p0=p0_rlc, maxfev=5000)
    A_fit, tau_fit, omega_fit, phi_fit, B_fit = popt_rlc
    
    # R^2
    r2_rlc = calc_r_squared(u_rlc_fit, fit_func_rlc(t_rlc_fit, *popt_rlc))
    
    print(f"【RLC 充电拟合 (t <= {limit_t} ms)】")
    print(f"  tau: {tau_fit*1000:.4f} us, omega: {omega_fit*1000:.4f} rad/s, R^2: {r2_rlc:.6f}")

except Exception as e:
    print(f"充电拟合失败: {e}")
    popt_rlc = None


# --- 2. 放电过程 (1 -> 0) ---
# 推导：
# 当输入信号变为 0 时，电路方程为 LC d^2u/dt^2 + RC du/dt + u = 0
# 特征方程根为 s = -alpha +/- j*omega_d
# 其中 alpha = R/2L = 1/tau, omega_d = sqrt(1/LC - (R/2L)^2)
# 通解为 u(t) = A * e^(-alpha * t) * cos(omega_d * t - phi)
# 考虑到放电从 t_start 时刻开始，公式可写为：
# u(t) = A * e^(-(t - t_start)/tau) * cos(omega * (t - t_start) - phi)
# 稳态值为 0。

t_start_discharge = 1.265
# 筛选放电数据 (t > 1.265 且排除异常值)
mask_dis = (t_rlc > t_start_discharge) & (t_rlc < 10)
t_rlc_dis = t_rlc[mask_dis]
u_rlc_dis = u_rlc[mask_dis]

def fit_func_rlc_discharge(t, A, tau, omega, phi):
    # t 是绝对时间，减去 t_start_discharge 得到相对衰减时间
    return A * np.exp(-(t - t_start_discharge) / tau) * np.cos(omega * (t - t_start_discharge) - phi)

# 初始猜测
# A: 初始振幅约为 1V (从峰值开始衰减)
p0_dis = [1.0, tau_guess, omega_guess, 0]

# 拟合放电
try:
    popt_dis, _ = curve_fit(fit_func_rlc_discharge, t_rlc_dis, u_rlc_dis, p0=p0_dis, maxfev=5000)
    A_dis, tau_dis, omega_dis, phi_dis = popt_dis
    
    # R^2
    r2_dis = calc_r_squared(u_rlc_dis, fit_func_rlc_discharge(t_rlc_dis, *popt_dis))
    
    print(f"【RLC 放电拟合 (t > {t_start_discharge} ms)】")
    print(f"  tau: {tau_dis*1000:.4f} us, omega: {omega_dis*1000:.4f} rad/s, R^2: {r2_dis:.6f}")

except Exception as e:
    print(f"放电拟合失败: {e}")
    popt_dis = None

# Save results
results_summary["RLC_Underdamped"] = {
    "charge": {
        "tau": tau_fit if popt_rlc is not None else None,
        "omega": omega_fit if popt_rlc is not None else None,
        "phi": phi_fit if popt_rlc is not None else None,
        "r2": r2_rlc if popt_rlc is not None else None,
        "params": popt_rlc
    },
    "discharge": {
        "tau": tau_dis if popt_dis is not None else None,
        "omega": omega_dis if popt_dis is not None else None,
        "phi": phi_dis if popt_dis is not None else None,
        "r2": r2_dis if popt_dis is not None else None,
        "params": popt_dis
    }
}

【RLC 充电拟合 (t <= 1.265 ms)】
  tau: 226.1749 us, omega: 21834.1149 rad/s, R^2: 0.999610
【RLC 放电拟合 (t > 1.265 ms)】
  tau: 226.6444 us, omega: 22439.2092 rad/s, R^2: 0.999719


In [6]:
# --- RLC 临界阻尼与过阻尼拟合 ---

# 通用 R^2 计算
def calc_r_squared(y_true, y_pred):
    residuals = y_true - y_pred
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

# ==========================================
# 1. 临界阻尼 (Critical Damping)
# ==========================================
# 理论公式：
# 充电 (0 -> E): u(t) = E * [1 - (1 + t/tau) * e^(-t/tau)]
# 放电 (E -> 0): u(t) = E * (1 + t/tau) * e^(-t/tau)

def fit_func_crit_charge(t, tau, E):
    return E * (1 - (1 + t / tau) * np.exp(-t / tau))

def fit_func_crit_discharge(t, tau, E, t0):
    dt = t - t0
    return E * (1 + dt / tau) * np.exp(-dt / tau)

print("-" * 30)
print("【RLC 临界阻尼分析】")

# 数据准备
t_crit = RLC_data2.t
u_crit = RLC_data2.UC
t_half_crit = 1.250 # 半周期分界点

# 拆分充放电
mask_charge = t_crit <= t_half_crit
t_crit_up = t_crit[mask_charge]
u_crit_up = u_crit[mask_charge]

mask_discharge = t_crit >= t_half_crit
t_crit_down = t_crit[mask_discharge]
u_crit_down = u_crit[mask_discharge]

# 拟合充电
# 只有3个点，参数少一点比较好。E 固定为 1.0 或者作为参数
try:
    popt_crit_up, _ = curve_fit(fit_func_crit_charge, t_crit_up, u_crit_up, p0=[0.1, 1.0])
    tau_crit_up, E_crit_up = popt_crit_up
    print(f"  [充电] tau: {tau_crit_up*1000:.4f} us, E: {E_crit_up:.4f} V")
except Exception as e:
    print(f"  [充电] 拟合失败: {e}")
    popt_crit_up = None

# 拟合放电
try:
    # 固定 t0
    fit_func_down_fixed = lambda t, tau, E: fit_func_crit_discharge(t, tau, E, t_half_crit)
    popt_crit_down, _ = curve_fit(fit_func_down_fixed, t_crit_down, u_crit_down, p0=[0.1, 1.0])
    tau_crit_down, E_crit_down = popt_crit_down
    print(f"  [放电] tau: {tau_crit_down*1000:.4f} us, E: {E_crit_down:.4f} V")
except Exception as e:
    print(f"  [放电] 拟合失败: {e}")
    popt_crit_down = None

# Save Critical results
results_summary["RLC_Critical"] = {
    "charge": {
        "tau": tau_crit_up if popt_crit_up is not None else None,
        "E": E_crit_up if popt_crit_up is not None else None,
        "params": popt_crit_up
    },
    "discharge": {
        "tau": tau_crit_down if popt_crit_down is not None else None,
        "E": E_crit_down if popt_crit_down is not None else None,
        "params": popt_crit_down
    }
}

# ==========================================
# 2. 过阻尼 (Over Damping)
# ==========================================
# 理论公式 (通用形式)：
# u(t) = B + A * e^(-t/tau) * sinh(beta*t + phi)
# 充电: B=E, A使得u(0)=0
# 放电: B=0

def fit_func_over_charge(t, tau, beta, A, phi, B):
    return B + A * np.exp(-t / tau) * np.sinh(beta * t + phi)

def fit_func_over_discharge(t, tau, beta, A, phi, t0):
    dt = t - t0
    return A * np.exp(-dt / tau) * np.sinh(beta * dt + phi)

print("-" * 30)
print("【RLC 过阻尼分析】")

# 数据准备
t_over = RLC_data3.t
u_over = RLC_data3.UC
t_half_over = 1.250

mask_charge_over = t_over <= t_half_over
t_over_up = t_over[mask_charge_over]
u_over_up = u_over[mask_charge_over]

mask_discharge_over = t_over >= t_half_over
t_over_down = t_over[mask_discharge_over]
u_over_down = u_over[mask_discharge_over]

# 拟合充电
# 数据点极少(3个)，参数有5个，必须固定参数。
# 假设 B=E_max (0.974), phi=0 (或根据边界条件), 仅拟合 tau, beta, A
# 或者使用更简单的形式： u(t) = E * (1 - e^(-t/tau1) - ... ) 双指数形式
# 这里尝试固定 B 和 phi
E_target = 0.974
def fit_func_over_charge_simple(t, tau, beta, A):
    # 假设 phi 使得 t=0 时为 0 -> B + A * sinh(phi) = 0 -> sinh(phi) = -B/A
    # 这里直接用通用形式拟合，但限制参数
    return E_target + A * np.exp(-t / tau) * np.sinh(beta * t) # 假设 phi=0 可能不满足 u(0)=0，仅作近似

# 由于数据点太少，这里仅作演示性拟合，或者使用双指数衰减形式
# u(t) = E * (1 - (tau1*e^(-t/tau1) - tau2*e^(-t/tau2))/(tau1-tau2)) ? 
# 让我们尝试拟合 tau 和 beta，固定 A 和 B
try:
    # 初始猜测
    tau_guess = RLC_data3.tau_theo / 1000
    # 强制减少自由度，否则无法拟合
    # 这里仅绘制连线或简单拟合
    pass 
except:
    pass

# 鉴于数据点过少 (3个点)，直接拟合 4-5 个参数的过阻尼公式是不可能的。
# 我们改用绘图展示，并计算理论曲线进行对比。

def theoretical_over_curve(t, R, L, C, E):
    # alpha = R/2L, omega0 = 1/sqrt(LC)
    alpha = R / (2 * L * 1e-3) * 1e-3 # 1/ms
    omega0 = 1 / np.sqrt(L * 1e-3 * C * 1e-6) * 1e-3 # 1/ms
    
    if alpha <= omega0:
        return np.zeros_like(t) # 不是过阻尼
        
    beta = np.sqrt(alpha**2 - omega0**2)
    
    # 充电: u(t) = E * [ 1 - e^(-alpha*t) * (cosh(beta*t) + alpha/beta * sinh(beta*t)) ]
    return E * (1 - np.exp(-alpha * t) * (np.cosh(beta * t) + alpha / beta * np.sinh(beta * t)))

# Save Overdamped results (mostly theoretical/placeholders)
results_summary["RLC_Overdamped"] = {
    "note": "Data points insufficient for full fitting. Theoretical curves used for visualization."
}

------------------------------
【RLC 临界阻尼分析】
  [充电] tau: 41.7077 us, E: 1.0000 V
  [放电] tau: 47.6659 us, E: 1.0000 V
------------------------------
【RLC 过阻尼分析】


In [7]:
# --- 综合绘图 ---
fig = plt.figure(figsize=(10, 12))
gs = fig.add_gridspec(3, 2, height_ratios=[1, 1, 1])

# 文本框样式
props = dict(boxstyle="round", facecolor="white", alpha=0.8, edgecolor="gray")

# ==========================================
# Row 1: RC 暂态 (RC_data1, RC_data2)
# ==========================================

def plot_rc_subplot(ax, res, name, max_v):
    # 1. 原始数据点
    ax.scatter(res['data_obj'].t, res['data_obj'].UC, label="数据点", color="black", s=10, zorder=5)

    # 2. 充电拟合曲线
    t_plot_up = np.linspace(0, np.max(res['t_up']), 100)
    ax.plot(
        t_plot_up,
        fit_func_charging(t_plot_up, res['tau_up'], max_v),
        "r-",
        lw=2,
        label="充电部分拟合",
    )

    # 3. 放电拟合曲线
    t_plot_down = np.linspace(0, np.max(res['t_down_shifted']), 100)
    ax.plot(
        t_plot_down + res['t_down_start'],
        fit_func_discharging(t_plot_down, res['tau_down'], res['v0_down']),
        "b-",
        lw=2,
        label="放电部分拟合",
    )

    # 4. 添加公式和 R^2 文本框
    info_text = (
        f"充电:\n"
        f"$U = {max_v:.3f}(1 - e^{{-t/{res['tau_up']:.1f}}})$\n"
        f"$R^2 = {res['r2_up']:.4f}$\n\n"
        f"放电:\n"
        f"$U = {res['v0_down']:.3f}e^{{-(t-{res['t_down_start']:.0f})/{res['tau_down']:.1f}}}$\n"
        f"$R^2 = {res['r2_down']:.4f}$"
    )

    ax.text(
        0.53,
        0.3,
        info_text,
        transform=ax.transAxes,
        fontsize=10,
        verticalalignment="center",
        horizontalalignment="right",
        multialignment="left",
        bbox=props,
    )

    ax.set_title(f"RC暂态$U_C$拟合{name}", fontsize=16)
    ax.set_xlabel(r"t($\mu s$)", fontsize=14)
    ax.set_ylabel("$U_C$(V)", fontsize=14)
    ax.legend(loc="upper right", fontsize=12)
    ax.tick_params(labelsize=12)
    ax.grid(True, linestyle="--", alpha=0.5)

ax1 = fig.add_subplot(gs[0, 0])
plot_rc_subplot(ax1, rc1_res, "1", max_v=V_SOURCE)

ax2 = fig.add_subplot(gs[0, 1])
plot_rc_subplot(ax2, rc2_res, "2", 1.0)


# ==========================================
# Row 2: RLC 欠阻尼 (RLC_data1) - 跨两列
# ==========================================
ax3 = fig.add_subplot(gs[1, :])

# 原始数据
mask_plot = t_rlc < 10
ax3.scatter(t_rlc[mask_plot], u_rlc[mask_plot], label='数据点', color='black', s=10)

# 充电拟合曲线
if popt_rlc is not None:
    t_plot_charge = np.linspace(0, t_start_discharge, 100)
    ax3.plot(t_plot_charge, fit_func_rlc(t_plot_charge, *popt_rlc), 'r-', lw=2, label='充电部分拟合')

# 放电拟合曲线
if popt_dis is not None:
    t_plot_dis = np.linspace(t_start_discharge, 2.5, 100)
    ax3.plot(t_plot_dis, fit_func_rlc_discharge(t_plot_dis, *popt_dis), 'b-', lw=2, label='放电部分拟合')

# 添加公式文本框
info_text_rlc = ""
if popt_rlc is not None:
    info_text_rlc += (
        f"充电:\n"
        f"$U \\approx {B_fit:.2f} + ({A_fit:.2f})e^{{-t/{tau_fit:.2f}}} \\cos({omega_fit:.1f}t)$\n"
        f"$R^2 = {r2_rlc:.4f}$\n\n"
    )
if popt_dis is not None:
    info_text_rlc += (
        f"放电:\n"
        f"$U \\approx {A_dis:.2f}e^{{-(t-{t_start_discharge})/{tau_dis:.2f}}} \\cos({omega_dis:.1f}(t-{t_start_discharge}) - {phi_dis:.2f})$\n"
        f"$R^2 = {r2_dis:.4f}$"
    )

ax3.text(
    0.98,
    0.53,
    info_text_rlc,
    transform=ax3.transAxes,
    fontsize=10,
    verticalalignment="center",
    horizontalalignment="right",
    multialignment="left",
    bbox=props,
)

ax3.set_title('RLC欠阻尼暂态响应（全周期拟合）', fontsize=16)
ax3.set_xlabel('t(ms)', fontsize=14)
ax3.set_ylabel('$U_C$(V)', fontsize=14)
ax3.legend(loc='upper right', fontsize=12)
ax3.tick_params(labelsize=12)
ax3.grid(True, linestyle='--', alpha=0.5)


# ==========================================
# Row 3: RLC 临界阻尼 & 过阻尼
# ==========================================

# --- 临界阻尼 ---
ax4 = fig.add_subplot(gs[2, 0])
ax4.scatter(t_crit, u_crit, color='black', label='数据点', s=10)

# 绘制拟合/理论曲线
if popt_crit_up is not None:
    t_plot = np.linspace(0, t_half_crit, 100)
    ax4.plot(t_plot, fit_func_crit_charge(t_plot, *popt_crit_up), 'r-', label='充电部分拟合')
if popt_crit_down is not None:
    t_plot = np.linspace(t_half_crit, 2.5, 100)
    ax4.plot(t_plot, fit_func_crit_discharge(t_plot, *popt_crit_down, t_half_crit), 'b-', label='放电部分拟合')

ax4.set_title('RLC临界阻尼暂态响应（拟合）', fontsize=16)
ax4.set_xlabel('t(ms)', fontsize=14)
ax4.set_ylabel('$U_C$(V)', fontsize=14)
ax4.legend(loc='upper right', fontsize=12)
ax4.tick_params(labelsize=12)
ax4.grid(True, linestyle='--', alpha=0.5)

# --- 过阻尼 ---
ax5 = fig.add_subplot(gs[2, 1])
ax5.scatter(t_over, u_over, color='black', label='数据点', s=10)

# 绘制理论曲线
# 使用实验参数 R, L, C
R_over = RLC_data3.R
L_over = RLC_data3.L
C_over = RLC_data3.C
E_over = 1 # 实验峰值

t_plot_up = np.linspace(0, t_half_over, 100)
u_plot_up = theoretical_over_curve(t_plot_up, R_over, L_over, C_over, E_over)
ax5.plot(t_plot_up, u_plot_up, 'r--', label='理论曲线（充电）')

# 放电理论曲线 (近似)
t_plot_down = np.linspace(t_half_over, 2.5, 100)
u_plot_down_decay = E_over - theoretical_over_curve(t_plot_down - t_half_over, R_over, L_over, C_over, E_over)
ax5.plot(t_plot_down, u_plot_down_decay, 'b', label='理论曲线（放电）')

ax5.set_title('RLC过阻尼暂态响应（理论）', fontsize=16)
ax5.set_xlabel('t(ms)', fontsize=14)
ax5.set_ylabel('$U_C$(V)', fontsize=14)
ax5.legend(loc='upper right', fontsize=12)
ax5.tick_params(labelsize=12)
ax5.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
# plt.show()
fig.savefig("rc_rlc_transient_analysis.png", dpi=500)

In [8]:
# --- 输出所有拟合结果汇总 ---
import numpy as np

# Helper to handle None values safely
def fmt(val, format_spec=".4f", factor=1.0):
    if val is None:
        return "N/A"
    if isinstance(val, (int, float, np.number)):
        return f"{val * factor:{format_spec}}"
    return str(val)

print("="*115)
print(f"{'拟合结果汇总 (Results Summary)':^115}")
print("="*115)

# --- 1. RC Transient ---
if 'RC' in results_summary:
    print(f"\n{'【RC 暂态 (RC Transient)】':<115}")
    print("-" * 115)
    # Header
    print(f"{'Dataset':<12} | {'Process':<12} | {'Tau (us)':<15} | {'R^2':<12} | {'V0/Offset (V)':<15}")
    print("-" * 115)
    
    for name, key in [("RC_data1", "rc1"), ("RC_data2", "rc2")]:
        res = results_summary['RC'].get(key)
        if res:
            # Charge
            print(f"{name:<12} | {'Charge':<12} | {fmt(res['tau_up']):<15} | {fmt(res['r2_up']):<12} | {'-':<15}")
            # Discharge
            print(f"{'':<12} | {'Discharge':<12} | {fmt(res['tau_down']):<15} | {fmt(res['r2_down']):<12} | {fmt(res['v0_down']):<15}")
            print("-" * 115)

# --- 2. RLC Underdamped ---
if 'RLC_Underdamped' in results_summary:
    res = results_summary['RLC_Underdamped']
    print(f"\n{'【RLC 欠阻尼 (Underdamped)】':<115}")
    print("-" * 115)
    # Stored values are in ms. Let's display in ms.
    print(f"{'Process':<15} | {'Tau (ms)':<15} | {'Omega (rad/ms)':<18} | {'Phi (rad)':<15} | {'R^2':<12}")
    print("-" * 115)
    
    # Charge
    c = res.get('charge', {})
    print(f"{'Charge':<15} | {fmt(c.get('tau')):<15} | {fmt(c.get('omega')):<18} | {fmt(c.get('phi')):<15} | {fmt(c.get('r2')):<12}")
    
    # Discharge
    d = res.get('discharge', {})
    print(f"{'Discharge':<15} | {fmt(d.get('tau')):<15} | {fmt(d.get('omega')):<18} | {fmt(d.get('phi')):<15} | {fmt(d.get('r2')):<12}")
    print("-" * 115)

# --- 3. RLC Critical Damping ---
if 'RLC_Critical' in results_summary:
    res = results_summary['RLC_Critical']
    print(f"\n{'【RLC 临界阻尼 (Critical Damping)】':<115}")
    print("-" * 115)
    print(f"{'Process':<15} | {'Tau (ms)':<15} | {'E (V)':<15}")
    print("-" * 115)
    
    # Charge
    c = res.get('charge', {})
    print(f"{'Charge':<15} | {fmt(c.get('tau')):<15} | {fmt(c.get('E')):<15}")
    
    # Discharge
    d = res.get('discharge', {})
    print(f"{'Discharge':<15} | {fmt(d.get('tau')):<15} | {fmt(d.get('E')):<15}")
    print("-" * 115)

# --- 4. RLC Overdamped ---
if 'RLC_Overdamped' in results_summary:
    res = results_summary['RLC_Overdamped']
    print(f"\n{'【RLC 过阻尼 (Overdamped)】':<115}")
    print("-" * 115)
    print(f"Note: {res.get('note', '')}")
    print("-" * 115)

                                             拟合结果汇总 (Results Summary)                                              

【RC 暂态 (RC Transient)】                                                                                             
-------------------------------------------------------------------------------------------------------------------
Dataset      | Process      | Tau (us)        | R^2          | V0/Offset (V)  
-------------------------------------------------------------------------------------------------------------------
RC_data1     | Charge       | 107.2682        | 0.9999       | -              
             | Discharge    | 113.0574        | 0.9985       | 1.0060         
-------------------------------------------------------------------------------------------------------------------
RC_data2     | Charge       | 124.8907        | 0.9979       | -              
             | Discharge    | 89.8021         | 0.9916       | 0.8400         
------------------------

In [9]:
# --- 补充分析：RLC_data2 (临界/过阻尼) 的理论过阻尼模型对比 ---

# 用户指出 R^2 (10^6) 略大于 4L/C (8*10^5)，因此 RLC_data2 实际上处于过阻尼状态。
# 这里使用过阻尼的理论公式进行绘图对比。

# 重新定义理论函数以确保可用
def theoretical_over_curve_local(t, R, L, C, E):
    # alpha = R/2L, omega0 = 1/sqrt(LC)
    alpha = R / (2 * L * 1e-3) * 1e-3 # 1/ms
    omega0 = 1 / np.sqrt(L * 1e-3 * C * 1e-6) * 1e-3 # 1/ms
    
    # 即使 alpha <= omega0 (欠阻尼/临界)，此公式在复数域也成立，但 numpy 可能报错
    # 这里仅处理过阻尼情况
    if alpha <= omega0:
        # 临界阻尼近似
        if np.isclose(alpha, omega0):
             return E * (1 - (1 + alpha * t) * np.exp(-alpha * t))
        return np.zeros_like(t) 
        
    beta = np.sqrt(alpha**2 - omega0**2)
    
    # 充电: u(t) = E * [ 1 - e^(-alpha*t) * (cosh(beta*t) + alpha/beta * sinh(beta*t)) ]
    return E * (1 - np.exp(-alpha * t) * (np.cosh(beta * t) + alpha / beta * np.sinh(beta * t)))

fig_over, ax_over = plt.subplots(figsize=(8, 6))

# 1. 获取参数 (RLC_data2)
R_val = RLC_data2.R  # 1000 Ohm
L_val = RLC_data2.L  # 20 mH
C_val = RLC_data2.C  # 0.1 uF
E_val = RLC_data2.E  # 1.0 V

# 2. 验证过阻尼条件
val_R2 = R_val**2
val_4LC = 4 * (L_val * 1e-3) / (C_val * 1e-6)
print(f"检查过阻尼条件 (RLC_data2):")
print(f"  R^2 = {val_R2:.2e}")
print(f"  4L/C = {val_4LC:.2e}")
print(f"  是否满足 R^2 > 4L/C? {'是 (过阻尼)' if val_R2 > val_4LC else '否'}")

# 3. 绘制数据点
ax_over.scatter(RLC_data2.t, RLC_data2.UC, color='black', s=30, zorder=5)

# 4. 绘制理论曲线
# 充电 (0 -> 1.25 ms)
t_half = 1.250
t_plot_up = np.linspace(0, t_half, 200)
u_plot_up = theoretical_over_curve_local(t_plot_up, R_val, L_val, C_val, E_val)
ax_over.plot(t_plot_up, u_plot_up, 'r-', lw=2)

# 放电 (1.25 -> 2.5 ms)
# 放电过程: u_dis(t) = E - u_charge(t) (从稳态E开始放电)
t_plot_down = np.linspace(t_half, 2.5, 200)
dt = t_plot_down - t_half
u_plot_down = E_val - theoretical_over_curve_local(dt, R_val, L_val, C_val, E_val)
ax_over.plot(t_plot_down, u_plot_down, 'b-', lw=2)

# 5. 装饰
# ax_over.set_title(f'RLC 临界/过阻尼数据与理论过阻尼曲线对比\n($R={R_val}\\Omega$, $L={L_val}$mH, $C={C_val}\\mu$F)', fontsize=16)
ax_over.set_xlabel('t (ms)', fontsize=14)
ax_over.set_ylabel('$U_C$ (V)', fontsize=14)
ax_over.legend(fontsize=12)
ax_over.grid(True, linestyle='--', alpha=0.5)
ax_over.tick_params(labelsize=12)

# 添加文本说明
info_text = (
    f"$R^2 = {val_R2:.1e}$\n"
    f"$4L/C = {val_4LC:.1e}$\n"
    f"Condition: $R^2 > 4L/C$"
)
ax_over.text(0.95, 0.4, info_text, transform=ax_over.transAxes, 
             fontsize=12, verticalalignment='top', horizontalalignment='right',
             bbox=dict(boxstyle="round", facecolor="white", alpha=0.8, edgecolor="gray"))

plt.tight_layout()
# plt.show()
fig_over.savefig("rlc_overdamped_theoretical_comparison.png", dpi=500)

检查过阻尼条件 (RLC_data2):
  R^2 = 1.00e+06
  4L/C = 8.00e+05
  是否满足 R^2 > 4L/C? 是 (过阻尼)


In [10]:
# --- 使用通用公式 (考虑非零初始条件) 进行拟合 ---
# 公式: U(t) = E + (V0 - E) * exp(-t/tau)
# 其中 E 为稳态值 (终值), V0 为初始值

def fit_func_general(t, tau, E, V0):
    return E + (V0 - E) * np.exp(-t / tau)

print("="*90)
print("通用公式拟合 (General Formula Fitting)")
print("公式: U(t) = E + (V0 - E) * exp(-t/tau)")
print("="*90)

def analyze_general(data_obj, name):
    # --- 充电 ---
    t_up = data_obj.t_up
    u_up = data_obj.UC_up
    
    # 初始猜测: tau=100, E=max(u), V0=min(u)
    p0_up = [100, np.max(u_up), np.min(u_up)]
    
    try:
        popt_up, _ = curve_fit(fit_func_general, t_up, u_up, p0=p0_up, maxfev=10000)
        tau_up, E_up, V0_up = popt_up
        r2_up = calc_r_squared(u_up, fit_func_general(t_up, *popt_up))
    except:
        tau_up, E_up, V0_up, r2_up = None, None, None, None

    # --- 放电 ---
    t_down = data_obj.t_down
    u_down = data_obj.UC_down
    t_down_shifted = t_down - t_down[0]
    
    # 放电时，初始值是 V0 (高), 稳态是 E (低, 趋向0)
    p0_down = [100, 0, np.max(u_down)]
    
    try:
        popt_down, _ = curve_fit(fit_func_general, t_down_shifted, u_down, p0=p0_down, maxfev=10000)
        tau_down, E_down, V0_down = popt_down
        r2_down = calc_r_squared(u_down, fit_func_general(t_down_shifted, *popt_down))
    except:
        tau_down, E_down, V0_down, r2_down = None, None, None, None
        
    return {
        "name": name,
        "charge": (tau_up, E_up, V0_up, r2_up),
        "discharge": (tau_down, E_down, V0_down, r2_down)
    }

# 分析
res1 = analyze_general(RC_data1, "RC_data1")
res2 = analyze_general(RC_data2, "RC_data2")

# 输出表格
print(f"{'Dataset':<10} | {'Process':<10} | {'Tau (us)':<12} | {'E (Steady) (V)':<18} | {'V0 (Start) (V)':<18} | {'R^2':<10}")
print("-" * 90)

for res in [res1, res2]:
    # Charge
    tau, E, V0, r2 = res['charge']
    print(f"{res['name']:<10} | {'Charge':<10} | {fmt(tau):<12} | {fmt(E):<18} | {fmt(V0):<18} | {fmt(r2):<10}")
    
    # Discharge
    tau, E, V0, r2 = res['discharge']
    print(f"{'':<10} | {'Discharge':<10} | {fmt(tau):<12} | {fmt(E):<18} | {fmt(V0):<18} | {fmt(r2):<10}")
    print("-" * 90)

通用公式拟合 (General Formula Fitting)
公式: U(t) = E + (V0 - E) * exp(-t/tau)
Dataset    | Process    | Tau (us)     | E (Steady) (V)     | V0 (Start) (V)     | R^2       
------------------------------------------------------------------------------------------
RC_data1   | Charge     | 107.7808     | 1.0092             | -0.0021            | 0.9999    
           | Discharge  | 112.3611     | -0.0086            | 1.0242             | 0.9992    
------------------------------------------------------------------------------------------
RC_data2   | Charge     | 109.4455     | 0.9361             | -0.0016            | 1.0000    
           | Discharge  | 111.3309     | -0.1025            | 0.8405             | 0.9996    
------------------------------------------------------------------------------------------


In [11]:
# --- 绘制通用公式拟合结果 ---
fig_gen, axes_gen = plt.subplots(1, 2, figsize=(12, 5))

def plot_general_fit(ax, data_obj, res, title):
    # 1. 原始数据
    ax.scatter(data_obj.t, data_obj.UC, color='black', label='Data', s=15, zorder=5)
    
    # 2. 充电拟合
    tau_up, E_up, V0_up, _ = res['charge']
    if tau_up is not None:
        t_plot_up = np.linspace(0, np.max(data_obj.t_up), 100)
        u_plot_up = fit_func_general(t_plot_up, tau_up, E_up, V0_up)
        ax.plot(t_plot_up, u_plot_up, 'r-', lw=2, label='Charge Fit')
        
    # 3. 放电拟合
    tau_down, E_down, V0_down, _ = res['discharge']
    if tau_down is not None:
        t_down_start = data_obj.t_down[0]
        t_plot_down_shifted = np.linspace(0, np.max(data_obj.t_down) - t_down_start, 100)
        u_plot_down = fit_func_general(t_plot_down_shifted, tau_down, E_down, V0_down)
        ax.plot(t_plot_down_shifted + t_down_start, u_plot_down, 'b-', lw=2, label='Discharge Fit')
        
    # 4. 装饰
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(r"t($\mu s$)", fontsize=12)
    ax.set_ylabel("$U_C$(V)", fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # 5. 添加参数文本
    info_text = (
        f"Charge:\n"
        f"$\\tau={tau_up:.1f}\\mu s$\n"
        f"$E={E_up:.3f}V, V_0={V0_up:.3f}V$\n\n"
        f"Discharge:\n"
        f"$\\tau={tau_down:.1f}\\mu s$\n"
        f"$E={E_down:.3f}V, V_0={V0_down:.3f}V$"
    )
    ax.text(0.95, 0.5, info_text, transform=ax.transAxes, 
            fontsize=9, verticalalignment='center', horizontalalignment='right',
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.8, edgecolor="gray"))

# 绘制两个子图
plot_general_fit(axes_gen[0], RC_data1, res1, "RC_data1 (General Fit)")
plot_general_fit(axes_gen[1], RC_data2, res2, "RC_data2 (General Fit)")

plt.tight_layout()
# plt.show()
fig_gen.savefig("rc_general_fit_comparison.png", dpi=500)

In [12]:
# --- 输出拟合公式 (保留6位小数) ---
print("="*100)
print("拟合公式汇总 (Fitted Formulas)")
print("="*100)

# 1. RC Standard
if 'RC' in results_summary:
    print("\n【RC 暂态 (标准模型)】")
    V_SOURCE_VAL = 1.006
    for key, name in [('rc1', 'RC_data1'), ('rc2', 'RC_data2')]:
        res = results_summary['RC'].get(key)
        if res:
            # Charge
            tau = res['tau_up']
            print(f"{name} 充电: U(t) = {V_SOURCE_VAL:.6f} * (1 - exp(-t / {tau:.6f}))")
            # Discharge
            tau = res['tau_down']
            v0 = res['v0_down']
            t_start = res['t_down_start']
            print(f"{name} 放电: U(t) = {v0:.6f} * exp(-(t - {t_start:.6f}) / {tau:.6f})")

# 2. RC General
if 'res1' in locals() and 'res2' in locals():
    print("\n【RC 暂态 (通用模型: U = E + (V0-E)exp(-t/tau))】")
    for res in [res1, res2]:
        name = res['name']
        # Charge
        tau, E, V0, _ = res['charge']
        if tau is not None:
             print(f"{name} 充电: U(t) = {E:.6f} + ({V0:.6f} - {E:.6f}) * exp(-t / {tau:.6f})")
        # Discharge
        tau, E, V0, _ = res['discharge']
        if tau is not None:
             data_obj = RC_data1 if res == res1 else RC_data2
             t_start = data_obj.t_down[0]
             print(f"{name} 放电: U(t) = {E:.6f} + ({V0:.6f} - {E:.6f}) * exp(-(t - {t_start:.6f}) / {tau:.6f})")

# 3. RLC Underdamped
if 'RLC_Underdamped' in results_summary:
    print("\n【RLC 欠阻尼】")
    res = results_summary['RLC_Underdamped']
    # Charge
    # params: A, tau, omega, phi, B
    p = res['charge']['params']
    if p is not None:
        A, tau, omega, phi, B = p
        print(f"充电: U(t) = {B:.6f} + ({A:.6f}) * exp(-t / {tau:.6f}) * cos({omega:.6f}t - {phi:.6f})")
    
    # Discharge
    # params: A, tau, omega, phi
    p = res['discharge']['params']
    if p is not None:
        A, tau, omega, phi = p
        t_start = 1.265 
        print(f"放电: U(t) = {A:.6f} * exp(-(t - {t_start:.6f}) / {tau:.6f}) * cos({omega:.6f}(t - {t_start:.6f}) - {phi:.6f})")

# 4. RLC Critical
if 'RLC_Critical' in results_summary:
    print("\n【RLC 临界阻尼】")
    res = results_summary['RLC_Critical']
    # Charge
    # params: tau, E
    p = res['charge']['params']
    if p is not None:
        tau, E = p
        print(f"充电: U(t) = {E:.6f} * [1 - (1 + t / {tau:.6f}) * exp(-t / {tau:.6f})]")
    
    # Discharge
    # params: tau, E
    p = res['discharge']['params']
    if p is not None:
        tau, E = p
        t_half = 1.250
        print(f"放电: U(t) = {E:.6f} * (1 + (t - {t_half:.6f}) / {tau:.6f}) * exp(-(t - {t_half:.6f}) / {tau:.6f})")

拟合公式汇总 (Fitted Formulas)

【RC 暂态 (标准模型)】
RC_data1 充电: U(t) = 1.006000 * (1 - exp(-t / 107.268191))
RC_data1 放电: U(t) = 1.006000 * exp(-(t - 632.000000) / 113.057387)
RC_data2 充电: U(t) = 1.006000 * (1 - exp(-t / 124.890720))
RC_data2 放电: U(t) = 0.840000 * exp(-(t - 251.000000) / 89.802138)

【RC 暂态 (通用模型: U = E + (V0-E)exp(-t/tau))】
RC_data1 充电: U(t) = 1.009181 + (-0.002137 - 1.009181) * exp(-t / 107.780793)
RC_data1 放电: U(t) = -0.008616 + (1.024188 - -0.008616) * exp(-(t - 632.000000) / 112.361069)
RC_data2 充电: U(t) = 0.936084 + (-0.001565 - 0.936084) * exp(-t / 109.445546)
RC_data2 放电: U(t) = -0.102474 + (0.840485 - -0.102474) * exp(-(t - 251.000000) / 111.330858)

【RLC 欠阻尼】
充电: U(t) = 1.013703 + (-1.017866) * exp(-t / 0.226175) * cos(21.834115t - 0.070435)
放电: U(t) = 0.988066 * exp(-(t - 1.265000) / 0.226644) * cos(22.439209(t - 1.265000) - 0.112034)

【RLC 临界阻尼】
充电: U(t) = 1.000000 * [1 - (1 + t / 0.041708) * exp(-t / 0.041708)]
放电: U(t) = 1.000000 * (1 + (t - 1.250000) / 0.047666) * 